In [117]:
import tensorflow as tf
print(tf.__version__)  # This notebook runs on TensorFlow 1.15.x or earlier
tf_framework_version = tf.__version__

2.5.1


In [118]:
import ktrain

from sagemaker import get_execution_role
from sagemaker import Session
role = get_execution_role()
sess = Session()
bucket = sess.default_bucket()

import os
import tensorflow as tf
import tensorflow.keras as keras
#from keras.models import model_from_json

In [119]:
pr = ktrain.load_predictor('saved')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [120]:
pr.predict('Data to test Infosys')

'Z7'

In [121]:
import h5py
import numpy as np


In [122]:
import pickle
from tensorflow.keras.models import load_model
# loading preprocess and model file
features = pickle.load(open('saved/tf_model.preproc',
                            'rb'))
loaded_model = load_model('saved/tf_model.h5')
labels = ['Z7', 'Z9A','Z9B', 'Z9C']

In [123]:
features;loaded_model


In [124]:
from tensorflow.python.saved_model import builder as builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [125]:
if tf.executing_eagerly():
    tf.compat.v1.disable_eager_execution()

In [126]:
# Note: This directory structure will need to be followed 
model_version = '1'
export_dir = 'export/Servo/' + model_version


# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
   inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

session = tf.compat.v1.Session()
init_op = tf.compat.v1.global_variables_initializer()
session.run(init_op)
# Save the meta graph and variables
builder.add_meta_graph_and_variables(
    sess=session, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
builder.save()

b'export/Servo/1/saved_model.pb'

In [127]:
model_path = 'export/Servo/1/'
#model_path = 
!saved_model_cli show --all --dir {model_path}


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 512)
        name: input_1_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['score'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 4)
        name: activation_1/Softmax:0
  Method name is: tensorflow/serving/predict


In [128]:
import tarfile
model_archive = 'model.tar.gz'
with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add('export', recursive=True) 

# upload model artifacts to S3
model_data = sess.upload_data(path=model_archive, key_prefix='model')

In [129]:
model_data

's3://sagemaker-eu-west-1-139008155365/model/model.tar.gz'

In [130]:
from sagemaker.tensorflow.serving import Model
instance_type = 'ml.c5.xlarge'

In [131]:
sm_model1 = Model(model_data=model_data, framework_version=tf_framework_version,role=role)
uncompiled_predictor_siv = sm_model1.deploy(initial_instance_count=1, instance_type=instance_type)

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

In [132]:
print(uncompiled_predictor_siv)

In [133]:
data = "KELLY SERVICES Title:Â DocumentÂ administrator TheÂ DocumentÂ administrator is theÂ associateÂ responsible for perform the technical (format) check of theÂ documents and its promotion to the Data Integrity or Final state depending on theÂ documentÂ type. Key responsibilities: - Checking all properties of theÂ documentÂ for accuracy with system properties , including eCTD filenames. - Responsible to support the DD&C (Device Development & Commercialization) team people with system issues, doubts providing clarification and resolution. - Checking theÂ documentfor compliance with IRDDS standards (International Research & DevelopmentÂ DocumentÂ Standards) and Novstyle, as appropriate. - Promote or demoteÂ documentÂ in lifecycle, promote for Data integrity check and or Approve as defined by SOP , demote to create (author possession for update). - Change ownership ofÂ documents depending of the doc. lifecycle status. - Execute manual retirement of doc. old versions, upon request from user and monthly by taking a list from system - Maintain a tracker of activities executed by DA - Responsible for support the DD&C team as record coordinator (paper archiving). - Responsible for support the DD&C team shipping packages true DHL and others Language: Excellent skills in English language are required. Professional Experience: The successful candidate for this position will have strong experience with office word and excel applications. Must: Experience/knowledge in: - TEDI System or aÂ Documentum system like Subway - IRDDS Good to have: Experience with Novstyle"

In [137]:
data = np.fromstring(data, dtype=np.uint8,count=-1, sep='')
data = np.resize(1,512)
predictions = uncompiled_predictor_siv.predict(data)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if __name__ == '__main__':


In [138]:
np.argmax(predictions)

0

In [135]:
# uncompiled_predictor_siv.delete_endpoint()